In [ ]:
import sys
import numpy as np
import xarray as xr
from glob import glob
from scipy import interpolate
import matplotlib.pyplot as plt 

from dask.distributed import Client

plt.rcParams['text.usetex']    = True


def calc_perc_temp(src): 
    # number of valid nodes above minimum ice-thickness
    # NN = src.height.where(src.height < 10).count(dim=('nMesh_node'))#.count(dim=('coord_1', 'coord_2'))
    NN = src.height.where(src.height > 10).count(dim=('coord_1', 'coord_2'))

    # mask for determining where nodes are temperate 
    mask = (src['enthalpy_h'] > src['phase change enthalpy']) & (src.height > 10)
    # number of nodes where ethalpy exceeds phase chnage enthalpy
    NT = src.enthalpy_h.where(mask).count(dim=('coord_1', 'coord_2'))
    return NT/NN * 100

In [ ]:
# client = Client()

In [ ]:
src_fp = '/Volumes/thermal_ssd/glc1-a_dx_50_SS@_2000_dt_1.0_MB_-1.5__-1.0_OFF_Tma_-9.0__-7.0_prog_gridded.nc'

SS_2kya = xr.open_dataset(src_fp)

In [ ]:
def basal_temp(ds): 
    basal = ds.isel(coord_2=0)
    
    xr.where(basal.enthalpy_h >= basal['phase change enthalpy'], 1, 0).sum('coord_1')

In [ ]:
basal = SS_2kya.isel(coord_2=0)

In [ ]:
basal.depth > 10

In [ ]:
xr.where((basal.enthalpy_h >= basal['phase change enthalpy']) & (basal.depth > 10), 1, 0).sum('coord_1') / basal.coord_1.size

In [ ]:
Vol = SS_2kya.height.isel(coord_2=-1).integrate("coord_1") /\
      SS_2kya.height.isel(coord_2=-1).isel(t=0).integrate("coord_1")

SS_2kya['Vol']  = Vol.interpolate_na(dim="T_ma")
# interpolate the missing model runs
SS_2kya['Perc'] = calc_perc_temp(SS_2kya.isel(t=-1)).interpolate_na(dim="T_ma")

In [ ]:
# Modeled grid of parameter vals
T_mgrid, B_mgrid = np.meshgrid(SS_2kya.T_ma, SS_2kya.Delta_MB)

# Dense grid of parameter vals for visulaization
T_dgrid, B_dgrid = np.meshgrid(np.linspace(T_mgrid.min(),T_mgrid.max(),100), 
                               np.linspace(B_mgrid.min(),B_mgrid.max(),100))

In [ ]:
tck  = interpolate.bisplrep(B_mgrid, T_mgrid, SS_2kya['Perc'].values)
znew = interpolate.bisplev(B_dgrid[:,0], T_dgrid[0,:], tck)


fig, ax = plt.subplots(1,1, figsize=(6,4), 
                       constrained_layout=True)

im = ax.contourf(T_dgrid, B_dgrid, znew, levels=np.linspace(0, 80, 41))
s  = ax.scatter(T_mgrid, B_mgrid, s=0.1, c='k')

ax.set_title('glc1-a')
ax.set_xlabel('$T_{\\rm ma}$ at $z_{\\rm ref}$  [$^\circ$C]')
ax.set_ylabel('$\Delta \dot{b}$ [m a$^{-1}$]')


cbar = fig.colorbar(im, drawedges=True)
# annotate the colorbar axes
cbar.set_label('\% Temperate [-]', rotation=270, labelpad=20)

# fig.savefig('/Users/andrewnolan/Desktop/Perc.png',  dpi=300, bbox_inches='tight')

In [ ]:
tck  = interpolate.bisplrep(B_mgrid, T_mgrid, SS_2kya['Vol'].isel(t=-1).values)
znew = interpolate.bisplev(B_dgrid[:,0], T_dgrid[0,:], tck)


fig, ax = plt.subplots(1,1, figsize=(6,4), 
                       constrained_layout=True)

im = ax.contourf(T_dgrid, B_dgrid, znew, levels=np.linspace(0.7,1.3,61), cmap='RdBu')
s  = ax.scatter( T_mgrid, B_mgrid, s=0.1, c='k')


ax.contour(T_dgrid, B_dgrid, znew, levels=[0.99, 1.0, 1.01], linestyles=[':', '-', ':'], colors='k', linewidths=0.5)
# ax.contourf(T_dgrid, B_dgrid, znew, levels=[0.99, 1.01], cmap='binary', alpha=0.25)


ax.set_title('glc1-a')
ax.set_xlabel('$T_{\\rm ma}$ at $z_{\\rm ref}$  [$^\circ$C]')
ax.set_ylabel('$\Delta \dot{b}$ [m a$^{-1}$]')


cbar = fig.colorbar(im, ticks=np.linspace(0.7,1.3, 13), drawedges=True)
# annotate the colorbar axes
cbar.set_label('Relative Volume per unit Width [-]', rotation=270, labelpad=20)

# fig.savefig('/Users/andrewnolan/Desktop/Vol.png', 
#             dpi=300, bbox_inches='tight', 
#             facecolor='w')

In [ ]:
subset = SS_2kya.isel(T_ma=slice(0,None,5), Delta_MB=slice(0,None,4))

In [ ]:
fig, axes = plt.subplots(6, 5, figsize=(8.5, 6), 
                         sharex=True, sharey=True, 
                         constrained_layout=True)

for i in range(6): 
    δ_b = float(subset.Delta_MB[i])
    axes[i,  0].set_ylabel(f'$\Delta \dot{{b}}={δ_b}$')

    if i > 0: 
        j = i-1 
        T_ma = float(subset.T_ma[j])
        axes[-1, j].set_xlabel(f'$T_{{ma}} ={T_ma}$')
    
    for j in range(5):
        
        im = axes[i,j].pcolormesh(subset.X[:,::-1], 
                             subset.Z.isel(T_ma=j, Delta_MB=i, t=-1), 
                             subset['temperature'].isel(T_ma=j, Delta_MB=i, t=-1), 
                             vmin=-10, vmax=0.0, 
                             shading='gouraud')
        
        V = float(subset['Vol' ].isel(T_ma=j, Delta_MB=i, t=-1))
        P = float(subset['Perc'].isel(T_ma=j, Delta_MB=i))/100.0
        
        axes[i,j].annotate(f"$V'$ = {V:1.2f} [-]", 
                           xy=(0.05,0.1), 
                           xycoords='axes fraction', 
                           fontsize='small')
        # axes[i,j].annotate(f"$TIF$ = {P:.2f} [-]", 
        #                    xy=(0.60,0.60), 
        #                    xycoords='axes fraction')
        
        axes[i,j].get_xaxis().set_ticks([])
        axes[i,j].get_yaxis().set_ticks([])
        
        axes[i,j].spines.right.set_visible(False)
        axes[i,j].spines.top.set_visible(False)
        
        # for spine in axes[i, j].spines.values():
        #     spine.set_visible(False)

cbar = fig.colorbar(im, ax=axes) 

cbar.set_label('Homologous Temperature [$^\circ$C]', 
               rotation=270, labelpad=10)

axes[0,0].set_xlim(None, 6e3)
axes[0,0].set_ylim(1.75e3, None)

# plt.subplots_adjust(wspace=0.1, hspace=0.1)

# fig.savefig('/Users/andrewnolan/Desktop/glaciers.png', dpi=400, bbox_inches='tight', facecolor='w')

In [ ]:
SS_2kya['Vol'].sel(T_ma=-9.0, Delta_MB = -1.225)